In [41]:
import pandas as pd
import numpy as np
import nltk
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from tpot import TPOTClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, mean_squared_error
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, GRU, LSTM, Dense, Dropout
from keras.initializers import Constant
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import pickle as pkl
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import os

In [42]:
df=pd.read_csv("FullyLabelled/sentiment-emotion-labelled_Dell_tweets.csv", index_col="Index")
df.columns=["DateTime", "TweetID", "Text", "Username", "Sentiment", "SScore", "Emotion","EScore"]
df.head()

,DateTime,TweetID,Text,Username,Sentiment,SScore,Emotion,EScore
Index,,,,,,,,
0,2022-09-30 23:29:15+00:00,1575991191170342912,@Logitech @apple @Google @Microsoft @Dell @Len...,ManjuSreedaran,neutral,0.853283,anticipation,0.587121
1,2022-09-30 21:46:35+00:00,1575965354425131008,@MK_habit_addict @official_stier @MortalKombat...,MiKeMcDnet,neutral,0.519470,joy,0.886913
2,2022-09-30 21:18:02+00:00,1575958171423752203,"As @CRN celebrates its 40th anniversary, Bob F...",jfollett,positive,0.763791,joy,0.960347
3,2022-09-30 20:05:24+00:00,1575939891485032450,@dell your customer service is horrible especi...,daveccarr,negative,0.954023,anger,0.983203
4,2022-09-30 20:03:17+00:00,1575939359160750080,@zacokalo @Dell @DellCares @Dell give the man ...,heycamella,neutral,0.529170,anger,0.776124


In [43]:
df.drop(["DateTime", "TweetID", "Username"], axis=1, inplace=True)
df.head()

,Text,Sentiment,SScore,Emotion,EScore
Index,,,,,
0,@Logitech @apple @Google @Microsoft @Dell @Len...,neutral,0.853283,anticipation,0.587121
1,@MK_habit_addict @official_stier @MortalKombat...,neutral,0.519470,joy,0.886913
2,"As @CRN celebrates its 40th anniversary, Bob F...",positive,0.763791,joy,0.960347
3,@dell your customer service is horrible especi...,negative,0.954023,anger,0.983203
4,@zacokalo @Dell @DellCares @Dell give the man ...,neutral,0.529170,anger,0.776124


In [44]:
df.Sentiment.unique(), df.Emotion.unique()

(array(['neutral', 'positive', 'negative'], dtype=object),
 array(['anticipation', 'joy', 'anger', 'sadness', 'fear', 'optimism',
        'disgust', 'surprise'], dtype=object))

In [45]:
df.Sentiment.value_counts(), df.Emotion.value_counts()

(Sentiment
 negative    10556
 positive     7366
 neutral      7048
 Name: count, dtype: int64,
 Emotion
 anger           7520
 joy             6326
 anticipation    5171
 disgust         3000
 sadness         1328
 optimism        1225
 fear             366
 surprise          34
 Name: count, dtype: int64)

In [46]:
sdict={'neutral':1, 'positive':0, 'negative':2}
edict={'disgust':0,'anger':1, 'sadness':2, 'fear':3,'surprise':4, 'optimism':5, 'joy':6,'anticipation':7}
df.Sentiment=df.Sentiment.replace(sdict)
df.Emotion=df.Emotion.replace(edict)
df.head()

,Text,Sentiment,SScore,Emotion,EScore
Index,,,,,
0,@Logitech @apple @Google @Microsoft @Dell @Len...,1,0.853283,7,0.587121
1,@MK_habit_addict @official_stier @MortalKombat...,1,0.519470,6,0.886913
2,"As @CRN celebrates its 40th anniversary, Bob F...",0,0.763791,6,0.960347
3,@dell your customer service is horrible especi...,2,0.954023,1,0.983203
4,@zacokalo @Dell @DellCares @Dell give the man ...,1,0.529170,1,0.776124


In [47]:
def process_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text)
    return text

df.Text=df.Text.apply(process_text)

In [48]:
df.Text.head(20)

Index
0     @Logitech @apple @Google @Microsoft @Dell @Len...
1     @MK_habit_addict @official_stier @MortalKombat...
2     As @CRN celebrates its 40th anniversary, Bob F...
3     @dell your customer service is horrible especi...
4     @zacokalo @Dell @DellCares @Dell give the man ...
5     The screenshot is acting up from the website o...
6     @emijuju_ @Alienware @Dell @IntelGaming alien ...
7     COOKING STREAM DAY!!! Ty to @Alienware for spo...
8      @emijuju_ @Alienware @Dell @intel Beautiful 😍❤️😻
9     What's your biggest data management challenge?...
10    What does it take to delight stakeholders with...
11    @BigwillUK @Alienware @Dell Good idea. I'll pu...
12    @teksyndicate @Alienware @Dell Looks great, ho...
13    Steve Gorman takes you through Part 2 of Explo...
14    Indian's decision to give incentives to @Apple...
15    I am Steve, a Geek. I am the CEO of @TheGeekVi...
16      @siim_kuusik @Alienware @Dell Because 3840x1600
17         ummm @Dell ...your customer ser

In [49]:
def process_text2(text):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    processed_text = []

    words = word_tokenize(text)

    processed_words = []
    for word in words:
        if word not in stop_words:
            stemmed_word = stemmer.stem(word)
            lemmatized_word = lemmatizer.lemmatize(stemmed_word)
            processed_words.append(lemmatized_word)

    processed_text.append(' '.join(processed_words))

    return processed_text


In [50]:
df.Text=df.Text.apply(process_text2)
df.Text.head(20)

Index
0     [@ logitech @ appl @ googl @ microsoft @ dell ...
1     [@ mk_habit_addict @ official_sti @ mortalkomb...
2     [a @ crn celebr 40th anniversari , bob faletra...
3     [@ dell custom servic horribl especi agent sye...
4     [@ zacokalo @ dell @ dellcar @ dell give man p...
5     [the screenshot act websit @ googl @ dell lati...
6     [@ emijuju_ @ alienwar @ dell @ intelgam alien...
7     [cook stream day ! ! ! ty @ alienwar sponsor s...
8     [@ emijuju_ @ alienwar @ dell @ intel beauti 😍...
9     [what 's biggest data manag challeng ? • cloud...
10    [what take delight stakehold onlin experi ? th...
11    [@ bigwilluk @ alienwar @ dell good idea . i '...
12    [@ teksynd @ alienwar @ dell look great , hope...
13    [steve gorman take part 2 explor @ f5 big-ip v...
14    [indian 's decis give incent @ appl @ dell bri...
15    [i steve , geek . i ceo @ thegeekvillag . talk...
16    [@ siim_kuusik @ alienwar @ dell becaus 3840x1...
17                [ummm @ dell ... custom 

In [51]:
tweets = df['Text'].tolist()
tokenized_tweets = [tweet[0].split() for tweet in tweets]

modelw2v = Word2Vec(tokenized_tweets, vector_size=300, window=5, min_count=10, workers=8)
modelw2v.train(tokenized_tweets, total_examples=len(tokenized_tweets), epochs=20)

(7806934, 12538260)

In [52]:
modelw2v.wv.most_similar("ai")

[('intellig', 0.6653841137886047),
 ('analyt', 0.62935870885849),
 ('real-tim', 0.614263117313385),
 ('iot', 0.612145185470581),
 ('edgecomput', 0.6081886291503906),
 ('➡️', 0.6026829481124878),
 ('multicloud', 0.6022351384162903),
 ('workshop', 0.6001420021057129),
 ('digitaltransform', 0.5979998111724854),
 ('machinelearn', 0.590273916721344)]

In [53]:

X = df.Text
y = df.drop(['Text'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=df["Emotion"])


In [54]:
y_test.Emotion.value_counts(), y_train.Emotion.value_counts()

(Emotion
 1    752
 6    633
 7    517
 0    300
 2    133
 5    122
 3     37
 4      3
 Name: count, dtype: int64,
 Emotion
 1    6768
 6    5693
 7    4654
 0    2700
 2    1195
 5    1103
 3     329
 4      31
 Name: count, dtype: int64)

In [55]:
def sentence_to_vec(s, model):
    # return np.array([model.wv[word] for word in s[0].split() if word in modelw2v.wv.key_to_index])
    vecs = [model.wv[word] for word in s[0].split() if word in model.wv.key_to_index]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)
X_train_vec = [sentence_to_vec(s, modelw2v) for s in X_train]
X_test_vec = [sentence_to_vec(s, modelw2v) for s in X_test]

max_len = max(max(len(x) for x in X_train_vec), max(len(x) for x in X_test_vec))

X_train_pad = pad_sequences(X_train_vec, maxlen=max_len, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_vec, maxlen=max_len, dtype='float32', padding='post')


In [56]:
X_test_pad.shape, y_test.shape

((2497, 300), (2497, 4))

In [57]:
# models = {
#     'Voting Classifier': VotingClassifier(estimators=[
#         ('xg', XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0)),
#         ('lgbm', LGBMClassifier(verbose=0, random_state=42)),
#         ('hgb', HistGradientBoostingClassifier(random_state=42))
#     ], voting='hard'),
#     'Logistic Regression': LogisticRegression(verbose=0, max_iter=1000),
#     'Decision Tree': DecisionTreeClassifier(random_state=42),
#     'Random Forest': RandomForestClassifier(verbose=0, random_state=42),
#     'AdaBoost': AdaBoostClassifier(random_state=42),
#     'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0),
#     'CatBoost': CatBoostClassifier(verbose=0, random_seed=42),
#     'LightGBM': LGBMClassifier(verbose=0, random_state=42),
#     'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42)
#     # 'TPOT': TPOTClassifier(generations=5, population_size=50, verbosity=0, random_state=42)
# }

# y_trains = np.array(y_train.Sentiment).reshape(-1,)

# for model_name, model in models.items():
#     print(f"Training {model_name}...")
#     model.fit(X_train_pad, y_trains)
    
#     print(f"Evaluating {model_name}...")
#     y_pred = model.predict(X_test_pad)
#     print(classification_report(y_test.Sentiment, y_pred))


In [58]:
# vocab_size = len(modelw2v.wv.key_to_index) + 1
# vector_size = modelw2v.wv.vector_size

# embedding_matrix = np.zeros((vocab_size, vector_size))
# for word, i in modelw2v.wv.key_to_index.items():
#     embedding_matrix[i] = modelw2v.wv[word]

In [59]:
# def create_model(model_type):
#     model = Sequential()
#     model.add(Embedding(vocab_size, vector_size, embeddings_initializer=Constant(embedding_matrix), trainable=True))
#     model.add(Dropout(0.5))
#     if model_type == 'RNN':
#         model.add(SimpleRNN(50))
#     elif model_type == 'GRU':
#         model.add(GRU(50))
#     elif model_type == 'LSTM':
#         model.add(LSTM(50))
    
#     model.add(Dense(1, activation='sigmoid'))
    
#     return model

In [60]:
# for model_type in ['RNN', 'GRU', 'LSTM']:
#     print(f"Training {model_type} model...")
#     model = create_model(model_type)
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model.fit(X_train_pad, y_trains, epochs=10, verbose=1)


In [61]:
# modele = {
#     'Logistic Regression': LogisticRegression(verbose=0, max_iter=1000),
#     'Decision Tree': DecisionTreeClassifier(random_state=42),
#     'Random Forest': RandomForestClassifier(verbose=0, random_state=42),
#     'AdaBoost': AdaBoostClassifier(random_state=42),
#     'Voting Classifier': VotingClassifier(estimators=[
#         ('xg', XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0)),
#         ('lgbm', LGBMClassifier(verbose=0, random_state=42)),
#         ('hgb', HistGradientBoostingClassifier(random_state=42))
#     ], voting='hard'),
#     'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0),
#     'CatBoost': CatBoostClassifier(verbose=0, random_seed=42),
#     'LightGBM': LGBMClassifier(verbose=0, random_state=42),
#     'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42)
#     # 'TPOT': TPOTClassifier(generations=5, population_size=50, verbosity=0, random_state=42)
# }

# y_trains = np.array(y_train.Emotion).reshape(-1,)

# for model_name, model in modele.items():
#     print(f"Training {model_name}...")
#     model.fit(X_train_pad, y_trains)
    
#     print(f"Evaluating {model_name}...")
#     y_pred = model.predict(X_test_pad)
#     print(classification_report(y_test.Emotion, y_pred, zero_division=0))

In [62]:
# modele = {
#     'Linear Regression': LinearRegression(),
#     'Decision Tree Regressor': DecisionTreeRegressor(random_state=42),
#     'Random Forest Regressor': RandomForestRegressor(random_state=42),
#     'AdaBoost Regressor': AdaBoostRegressor(random_state=42),
#     'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
#     'XGBoost Regressor': XGBRegressor(use_label_encoder=False, eval_metric='logloss', verbosity=0),
#     'CatBoost Regressor': CatBoostRegressor(verbose=0, random_seed=42),
#     'LightGBM Regressor': LGBMRegressor(random_state=42)
# }

# y_trains = np.array(y_train.SScore).reshape(-1,)

# for model_name, model in modele.items():
#     print(f"Training {model_name}...")
#     model.fit(X_train_pad, y_trains)
    
#     print(f"Evaluating {model_name}...")
#     y_pred = model.predict(X_test_pad)
#     print('Mean Squared Error:', mean_squared_error(y_test.SScore, y_pred))


In [63]:
# modele = {
#     'Linear Regression': LinearRegression(),
#     'Decision Tree Regressor': DecisionTreeRegressor(random_state=42),
#     'Random Forest Regressor': RandomForestRegressor(random_state=42),
#     'AdaBoost Regressor': AdaBoostRegressor(random_state=42),
#     'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
#     'XGBoost Regressor': XGBRegressor(use_label_encoder=False, eval_metric='logloss', verbosity=0),
#     'CatBoost Regressor': CatBoostRegressor(verbose=0, random_seed=42),
#     'LightGBM Regressor': LGBMRegressor(random_state=42)
# }

# y_trains = np.array(y_train.EScore).reshape(-1,)

# for model_name, model in modele.items():
#     print(f"Training {model_name}...")
#     model.fit(X_train_pad, y_trains)
    
#     print(f"Evaluating {model_name}...")
#     y_pred = model.predict(X_test_pad)
#     print('Mean Squared Error:', mean_squared_error(y_test.EScore, y_pred))


In [64]:
# os.system(f'cp {"FullyLabelled/sentiment-emotion-labelled_Dell_tweets.csv"} {"TempSmall.csv"}')
# os.system(f'cp {"SemiLabelled/training.1600000.processed.noemoticon.csv"} {"TempLarge.csv"}')

0

In [65]:
# small=pd.read_csv("TempSmall.csv", index_col="Index")
# small.columns=["DateTime", "TweetID", "Text", "Username", "Sentiment", "SScore", "Emotion","EScore"]
# small.drop(["DateTime", "TweetID", "Username"], axis=1, inplace=True)
# small.Sentiment=small.Sentiment.replace(sdict)
# small.Emotion=small.Emotion.replace(edict)
# small.Text=small.Text.apply(process_text)
# small.Text=small.Text.apply(process_text2)

# large=pd.read_csv("TempLarge.csv", names=["Sentiment", "ID2", "DT", "Q", "UserName", "Text"], encoding="ISO-8859-1")
# large.drop(["DT", "ID2", "UserName", "Q"], axis=1, inplace=True)
# large.Sentiment=large.Sentiment.replace({0:2, 4:0,2:1})
# large.Text=large.Text.apply(process_text)
# large.Text=large.Text.apply(process_text2)

# small.to_csv("TempSmall.csv")
# large.to_csv("TempLarge.csv")

# tweets = small['Text'].tolist()+large['Text'].tolist()
# tokenized_tweets = [tweet[0].split() for tweet in tweets]

# modelw2v = Word2Vec(tokenized_tweets, vector_size=300, window=5, min_count=10, workers=8)
# modelw2v.train(tokenized_tweets, total_examples=len(tokenized_tweets), epochs=20)

# pkl.dump(modelw2v, open("W2V_Model.pkl", "wb"))


In [66]:

X_train_vec = [sentence_to_vec(s, modelw2v) for s in small.Text]
X_test_vec = [sentence_to_vec(s, modelw2v) for s in large.Text]

max_len = max(max(len(x) for x in X_train_vec), max(len(x) for x in X_test_vec))

X_train_pad = pad_sequences(X_train_vec, maxlen=max_len, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_vec, maxlen=max_len, dtype='float32', padding='post')

y_train=small.drop(["Text"])
y_test=large.drop(["Text"])


: 

In [ ]:
def catboost_optimization(cv_splits, X, y, mode=1):
    def function(learning_rate, depth, l2_leaf_reg, border_count):
        if mode:
            return cross_val_score(
                CatBoostClassifier(
                    learning_rate=learning_rate,
                    depth=int(round(depth)),
                    l2_leaf_reg=l2_leaf_reg,
                    border_count=int(round(border_count)),
                    loss_function="Logloss",
                    verbose=False,
                ),
                X=X,
                y=y,
                cv=cv_splits,
                scoring="accuracy",
                n_jobs=-1,
            ).mean()
        else:
            return cross_val_score(
                CatBoostRegressor(
                    learning_rate=learning_rate,
                    depth=int(round(depth)),
                    l2_leaf_reg=l2_leaf_reg,
                    border_count=int(round(border_count)),
                    loss_function="Logloss",
                    verbose=False,
                ),
                X=X,
                y=y,
                cv=cv_splits,
                scoring="accuracy",
                n_jobs=-1,
            ).mean()

    return function




In [ ]:
while len(X_test_pad) > 0:
    models = {
        "Sentiment": CatBoostClassifier(verbose=False),
        "Emotion": CatBoostClassifier(verbose=False),
        "SScore": CatBoostRegressor(verbose=False),
        "EScore": CatBoostRegressor(verbose=False),
    }

    for label in models.keys():
        if label=="SScore" or label=="EScore":
            mode=0
        else:
            mode=1
        optimizer = BayesianOptimization(
            f=catboost_optimization(5, X_train_pad, y_train[label], mode),
            pbounds={
                "learning_rate": (0.01, 1.0),
                "depth": (4, 10),
                "l2_leaf_reg": (1, 10),
                "border_count": (5, 255)
            },
            random_state=1234,
            verbose=2
        )

        optimizer.maximize(n_iter=25, init_points=2)
        params=optimizer.max["params"]
        if mode:
            m=CatBoostClassifier(**params)
        else:
            m=CatBoostRegressor(**params)
        
        m.fit(X_train_pad, y_train[label])
        
        models[label]=m

    
    predictions = {}
    for label in models.keys():
        predictions[label] = models[label].predict_proba(X_test_pad)

    indices = np.where(
        (predictions["Sentiment"][:, 1] > 0.9)
        & (predictions["Emotion"][:, 1] > 0.9)
        & (predictions["SScore"][:, 1] > 0.9)
        & (predictions["EScore"][:, 1] > 0.9)
    )

    X_train_pad = np.concatenate((X_train_pad, X_test_pad[indices]))
    y_train["Sentiment"] = np.concatenate(
        (y_train["Sentiment"], predictions["Sentiment"][indices, 1])
    )
    y_train["Emotion"] = np.concatenate(
        (y_train["Emotion"], predictions["Emotion"][indices, 1])
    )
    y_train["SScore"] = np.concatenate(
        (y_train["SScore"], predictions["SScore"][indices, 1])
    )
    y_train["EScore"] = np.concatenate(
        (y_train["EScore"], predictions["EScore"][indices, 1])
    )

    X_test_pad = np.delete(X_test_pad, indices, axis=0)
    y_test=np.delete(y_test, indices, axis=0)
    

In [ ]:
for label in models.keys():
        models[label].fit(X_train_pad, y_train[label])